In [ ]:
# ============================================================================
# PACKAGE INSTALLATION
# ============================================================================
# Install the backtesting library for financial strategy backtesting
!pip install backtesting

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for backtesting from https://files.pythonhosted.org/packages/b3/b6/cf57538b968c5caa60ee626ec8be1c31e420067d2a4cf710d81605356f8c/backtesting-0.6.5-py3-none-any.whl.metadata
  Obtaining dependency information for bokeh!=3.0.*,!=3.2.*,>=3.0.0 from https://files.pythonhosted.org/packages/91/48/08b2382e739236aa3360b7976360ba3e0c043b6234e25951c18c1eb6fa06/bokeh-3.7.3-py3-none-any.whl.metadata
  Obtaining dependency information for contourpy>=1.2 from https://files.pythonhosted.org/packages/98/4b/9bd370b004b5c9d8045c6c33cf65bae018b27aca550a3f657cdc99acdbd8/contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for narwhals>=1.13 from https://files.pythonhosted.org/packages/7f/26/43caf834e47c63883a5eddc02893b7fdbe6a0a4508ff6dc401907f3cc085/narwhals-2.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/192.1 kB ? eta -:--:--
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 1.2.3 requires bokeh<3.3.0,>=3.1.1, but you have bokeh 3.7.3 which is incompatible.


In [ ]:
# ============================================================================
# VWAP BREAKOUT STRATEGY - INTRADAY BACKTESTING
# ============================================================================
# A comprehensive implementation of Volume Weighted Average Price (VWAP) 
# breakout strategy for intraday trading using the backtesting framework
# ============================================================================

# Import required libraries
import pandas as pd
import numpy as np
from datetime import time
from backtesting import Backtest, Strategy

def add_vwap(
    df: pd.DataFrame,
    time_col: str | None = None,
    price_col: str = "Close",
    vol_col: str = "Volume",
    **parse_kwargs                # Additional arguments forwarded to pd.to_datetime
) -> pd.DataFrame:
    """
    Add daily VWAP (Volume Weighted Average Price) to a DataFrame.
    
    VWAP is calculated as the cumulative sum of (price * volume) divided by
    the cumulative sum of volume, reset at the beginning of each trading day.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame containing OHLCV data
    time_col : str | None
        Name of the timestamp column (if index is not datetime)
    price_col : str
        Price column to use for VWAP calculation (default: "Close")
        Use "Typical" for typical price (H+L+C)/3
    vol_col : str
        Volume column name (default: "Volume")
    **parse_kwargs
        Additional arguments passed to pd.to_datetime
        
    Returns:
    --------
    pd.DataFrame
        Copy of input DataFrame with added 'VWAP' column
    """
    df = df.copy()

    # Step 1: Ensure DatetimeIndex for proper time-based operations
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce", **parse_kwargs)
        df = df.set_index(time_col)

    if not isinstance(df.index, pd.DatetimeIndex) or df.index.hasnans:
        raise TypeError("Index (or `time_col`) must be datetime-like and parse without NaT")

    # Step 2: Calculate the price to weight by volume
    # Use typical price (H+L+C)/3 if specified, otherwise use the given price column
    tp = ((df["High"] + df["Low"] + df["Close"]) / 3) if price_col.lower() == "typical" else df[price_col]

    # Step 3: Calculate VWAP per calendar day
    # Group by normalized date (removes time component) to reset VWAP daily
    day = df.index.normalize()
    cum_vol = df[vol_col].groupby(day).cumsum()      # Cumulative volume per day
    cum_pv  = (tp * df[vol_col]).groupby(day).cumsum()  # Cumulative price*volume per day
    
    # VWAP = Cumulative (Price * Volume) / Cumulative Volume
    df["VWAP"] = cum_pv / cum_vol
    return df

C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
def add_weekly_vwap(
    df: pd.DataFrame,
    *,
    time_col: str | None = None,
    price_col: str = "Close",
    vol_col:   str = "Volume",
    freq:      str = "D",           # "D" (daily) or "W-MON", "W-FRI", etc.
    **parse_kwargs,                 # Additional arguments forwarded to pd.to_datetime
) -> pd.DataFrame:
    """
    Add VWAP column that resets every specified frequency period.
    
    This function allows for flexible VWAP calculation periods beyond just daily,
    such as weekly, monthly, or custom time periods. Useful for longer-term
    VWAP analysis and strategy development.

    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame containing OHLCV data
        Must contain `price_col`, `vol_col`, and optionally High/Low if price_col="Typical"
    time_col : str | None
        Name of the timestamp column if the index is not already DateTime-like
    price_col : {"Close", "Typical", ...}
        Price column to weight by volume
        "Typical" uses (High+Low+Close)/3 for more representative pricing
    vol_col : str
        Volume column name for weighting calculations
    freq : str, default "D"
        Pandas offset alias defining VWAP reset boundary
        Examples: 
        - "D" (daily reset)
        - "W-MON" (weekly ending Monday)
        - "W-FRI" (weekly ending Friday) 
        - "M" (monthly reset)
    **parse_kwargs
        Extra arguments passed to `pd.to_datetime` when parsing `time_col`

    Returns:
    --------
    pd.DataFrame
        Copy of input DataFrame with an added 'VWAP' column
        
    Examples:
    ---------
    # Weekly VWAP ending on Fridays
    df_weekly = add_weekly_vwap(df, freq="W-FRI")
    
    # Monthly VWAP
    df_monthly = add_weekly_vwap(df, freq="M")
    """
    df = df.copy()

    # Step 1: Ensure DatetimeIndex for time-based grouping operations
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce", **parse_kwargs)
        df = df.set_index(time_col)

    if not isinstance(df.index, pd.DatetimeIndex) or df.index.hasnans:
        raise TypeError("Index (or `time_col`) must be datetime-like and parse without NaT")

    # Step 2: Choose the appropriate price series for VWAP calculation
    if price_col.lower() == "typical":
        # Typical price provides better representation of intraday price action
        tp = (df["High"] + df["Low"] + df["Close"]) / 3
    else:
        tp = df[price_col]

    # Step 3: Create labels defining each VWAP reset period
    # Convert to period and back to timestamp to floor to period start
    # Example: if freq="W-MON", floors to Monday 00:00 of each week
    labels = df.index.to_period(freq).to_timestamp()

    # Step 4: Calculate cumulative sums within each defined period
    cum_vol = df[vol_col].groupby(labels).cumsum()        # Cumulative volume per period
    cum_pv  = (tp * df[vol_col]).groupby(labels).cumsum()   # Cumulative price*volume per period

    # Step 5: Calculate VWAP = Cumulative (Price * Volume) / Cumulative Volume
    df["VWAP"] = cum_pv / cum_vol
    return df

In [ ]:
# ============================================================================
# VWAP BREAKOUT TRADING STRATEGY
# ============================================================================
# Implementation of a systematic VWAP breakout strategy with the following rules:
# - LONG: When price closes above VWAP AND above day's opening price
# - SHORT: When price closes below VWAP AND below day's opening price
# - Risk Management: Optional ATR-based trailing stop loss
# - Intraday Exit: All positions closed before market close (15:45 ET)
# ============================================================================

class VWAPBreakout(Strategy):
    """
    VWAP Breakout Strategy Implementation
    
    Trading Logic:
    - Entry Long: Close > VWAP AND Close > Day's Open (bullish bias filter)
    - Entry Short: Close < VWAP AND Close < Day's Open (bearish bias filter)
    - Exit: Positions automatically closed before market close (15:45 ET)
    - Risk Management: Optional ATR-based trailing stop loss
    
    The strategy combines VWAP breakout signals with daily bias filtering
    to improve signal quality and reduce false breakouts.
    """

    # ========================================================================
    # CONFIGURABLE STRATEGY PARAMETERS
    # ========================================================================
    # These parameters can be optimized during backtesting
    
    intraday_close_time = time(15, 45)  # Time to close all positions (15:45 ET)
                                       # Ensures flat position before market close
    
    atr_stop = 1.5                     # ATR multiplier for trailing stop loss
                                       # Set to None to disable stop loss
                                       # Higher values = wider stops, lower values = tighter stops

    def init(self):
        """
        Initialize strategy indicators and setup.
        
        This method is called once at the beginning of backtesting
        to pre-compute any required technical indicators.
        """
        # Pre-compute Average True Range (ATR) if trailing stop is enabled
        if self.atr_stop:
            # ATR is used to set dynamic stop losses based on market volatility
            self.atr = self.I(self._atr, self.data.High, self.data.Low, self.data.Close, 14)

    @staticmethod
    def _atr(h, l, c, n):
        """
        Calculate Average True Range (ATR) for volatility-based stop losses.
        
        ATR measures market volatility and helps set appropriate stop loss levels
        that adapt to current market conditions.
        
        Parameters:
        -----------
        h : array-like
            High prices
        l : array-like  
            Low prices
        c : array-like
            Close prices
        n : int
            Lookback period for ATR calculation (typically 14)
            
        Returns:
        --------
        numpy.array
            ATR values aligned with input data length
        """
        # True Range calculation: max of three values
        # 1. High - Low (current period range)
        # 2. |High - Previous Close| (gap up scenarios)  
        # 3. |Low - Previous Close| (gap down scenarios)
        tr = np.maximum.reduce([
            h[1:] - l[1:],           # Current period High - Low
            abs(h[1:] - c[:-1]),     # Current High - Previous Close
            abs(l[1:] - c[:-1])      # Current Low - Previous Close
        ])
        
        # Calculate rolling average of True Range
        atr = pd.Series(tr).rolling(n).mean()
        
        # Prepend NaN to align with original data length
        return np.append([np.nan], atr)

    def next(self):
        """
        Strategy logic executed on each new price bar.
        
        This method contains the core trading logic including:
        - Entry signal detection
        - Position management
        - Risk management (trailing stops)
        - Intraday exit rules
        """
        # ====================================================================
        # GET CURRENT MARKET DATA
        # ====================================================================
        close = self.data.Close[-1]    # Current close price
        vwap  = self.data.VWAP[-1]     # Current VWAP value

        # ====================================================================
        # CALCULATE DAILY BIAS FILTER
        # ====================================================================
        # Get the opening price for the current trading day
        # This is used as a bias filter to improve signal quality
        current_day = self.data.index[-1].date()
        day_open = self.data.Open[self.data.index.date == current_day][0]

        # ====================================================================
        # ENTRY LOGIC - ONLY WHEN NO POSITION IS HELD
        # ====================================================================
        if not self.position:

            # LONG ENTRY CONDITIONS
            # 1. Price closes above VWAP (primary signal)
            # 2. Price closes above day's open (bullish bias filter)
            if (close > vwap) and (close > day_open):
                self.buy()
                
            # SHORT ENTRY CONDITIONS  
            # 1. Price closes below VWAP (primary signal)
            # 2. Price closes below day's open (bearish bias filter)
            elif (close < vwap) and (close < day_open):
                self.sell()

        # ====================================================================
        # RISK MANAGEMENT - ATR TRAILING STOP LOSS
        # ====================================================================
        if self.position and self.atr_stop:
            price = self.data.Close[-1]     # Current price for stop calculation
            atr = self.atr[-1]              # Current ATR value
            trail = self.atr_stop * atr     # Stop distance = ATR * multiplier

            # Update trailing stop for all active trades
            # Stop loss trails the price but never moves against the position
            for trade in self.trades:
                if trade.is_long:
                    # For long positions: stop trails below price
                    new_sl = price - trail
                    # Only update if new stop is higher (trailing up)
                    if trade.sl is None or new_sl > trade.sl:
                        trade.sl = new_sl
                else:
                    # For short positions: stop trails above price  
                    new_sl = price + trail
                    # Only update if new stop is lower (trailing down)
                    if trade.sl is None or new_sl < trade.sl:
                        trade.sl = new_sl

        # ====================================================================
        # INTRADAY EXIT RULE - CLOSE ALL POSITIONS BEFORE MARKET CLOSE
        # ====================================================================
        if self.position:
            bar_time = self.data.index[-1].time()
            
            # Close all positions at or after the specified time (15:45 ET)
            # This ensures we don't hold overnight positions (intraday strategy)
            if bar_time >= self.intraday_close_time:
                self.position.close()

In [ ]:
# ============================================================================
# DATA LOADING AND BACKTESTING EXECUTION
# ============================================================================
# Load market data, calculate VWAP, and run the backtesting simulation
# ============================================================================

# ────────────────────────────────────────────────────────────────────────────
# DATA LOADING OPTIONS
# ────────────────────────────────────────────────────────────────────────────
# Multiple data sources available - uncomment the desired dataset:

# Tesla 15-minute data (high volatility stock)
#df = pd.read_csv("TSLA.USUSD_Candlestick_15_M_BID_01.06.2024-28.06.2025.csv")

# Bitcoin 15-minute data (cryptocurrency)  
#df = pd.read_csv("BTCUSD_Candlestick_15_M_BID_01.06.2024-28.06.2025.csv")

# Apple daily data (longer timeframe testing)
#df = pd.read_csv("AAPL.USUSD_Candlestick_1_D_BID_01.06.2020-28.06.2025.csv")

# Current dataset - generic data file
df = pd.read_csv("data.csv")

# ────────────────────────────────────────────────────────────────────────────
# VWAP CALCULATION
# ────────────────────────────────────────────────────────────────────────────
# Add daily VWAP to the dataset using the helper function
df = add_vwap(
    df,
    time_col="Gmt time",                       # Timestamp column name
    dayfirst=True,                             # Parse dates as DD.MM.YYYY format
    format="%d.%m.%Y %H:%M:%S.%f"             # Exact datetime format for faster parsing
)

# ────────────────────────────────────────────────────────────────────────────
# ALTERNATIVE: WEEKLY VWAP CALCULATION  
# ────────────────────────────────────────────────────────────────────────────
# Uncomment below to use weekly VWAP instead of daily VWAP
# Weekly VWAP can provide different trading signals and may be more suitable
# for certain market conditions or longer-term strategies

# df = add_weekly_vwap(
#     df[100:],                              # Skip first 100 rows for stability
#     time_col="Gmt time",                   # Let the helper parse & index the timestamp
#     dayfirst=True,                         # Parse 03.06.2024 as 3 June 2024
#     format="%d.%m.%Y %H:%M:%S.%f",        # Exact pattern for faster parsing/no NaT
#     freq="W-FRI"                          # Weekly VWAP, week ends Friday
# )

# ────────────────────────────────────────────────────────────────────────────
# BACKTESTING SETUP AND EXECUTION
# ────────────────────────────────────────────────────────────────────────────
# Configure and run the backtesting simulation

bt = Backtest(
    df[1000:5000],                            # Data subset for backtesting (rows 1000-5000)
                                              # Using subset for faster execution and stable VWAP
    
    VWAPBreakout,                             # Strategy class to test
    
    cash=100_000,                             # Starting capital ($100,000)
    
    commission=0.000,                         # Commission per trade (0% for testing)
                                              # Set to realistic value e.g., 0.001 for 0.1%
    
    exclusive_orders=True,                    # Only one position at a time
                                              # Prevents multiple simultaneous positions
    
    trade_on_close=True,                      # Execute trades on bar close prices
                                              # More realistic than using open prices
)

# ────────────────────────────────────────────────────────────────────────────
# RUN BACKTEST AND DISPLAY RESULTS
# ────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt

# Execute the backtesting simulation
stats = bt.run()

# Display comprehensive performance statistics
print("=" * 60)
print("BACKTESTING RESULTS - VWAP BREAKOUT STRATEGY")
print("=" * 60)
print(stats)

# Generate interactive plot showing:
# - Price chart with entry/exit points
# - Equity curve
# - Drawdown periods
# - Trade markers
bt.plot(show_legend=False)                    # Hide legend for cleaner visualization

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_17488\643054199.py:30: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


Start                     2022-08-24 16:30:00
End                       2023-04-05 18:30:00
Duration                    224 days 02:00:00
Exposure Time [%]                        81.3
Equity Final [$]                 238235.14314
Equity Peak [$]                  240909.57314
Return [%]                          138.23514
Buy & Hold Return [%]               -36.67283
Return (Ann.) [%]                   310.14602
Volatility (Ann.) [%]               192.55241
CAGR [%]                            165.44456
Sharpe Ratio                          1.61071
Sortino Ratio                        13.52319
Calmar Ratio                         21.95156
Alpha [%]                           138.33854
Beta                                  0.00282
Max. Drawdown [%]                   -14.12866
Avg. Drawdown [%]                    -1.91476
Max. Drawdown Duration       28 days 00:15:00
Avg. Drawdown Duration        2 days 13:42:00
# Trades                                 1284
Win Rate [%]                      

GridPlot(id='p1378', ...)

In [ ]:
# ============================================================================
# STRATEGY OPTIMIZATION - ATR STOP LOSS PARAMETER SWEEP
# ============================================================================
# Systematically test different ATR stop loss multipliers to find the optimal
# risk management setting that maximizes the Sharpe ratio
# ============================================================================

# ────────────────────────────────────────────────────────────────────────────
# OPTIMIZATION PARAMETERS
# ────────────────────────────────────────────────────────────────────────────
# Test ATR stop multipliers from 1.0 to 2.5 in 0.25 increments
# This range covers:
# - Tight stops (1.0x ATR): More frequent stops, lower drawdowns
# - Medium stops (1.5-2.0x ATR): Balanced approach  
# - Wide stops (2.5x ATR): Fewer stops, higher potential drawdowns

stats_best, heatmap = bt.optimize(
    atr_stop = [round(x, 2) for x in np.arange(1, 2.51, 0.25)],  # Parameter range to test
    maximize = "Sharpe Ratio",                                   # Optimization objective
    return_heatmap = True,                                       # Keep all results for analysis
)

# ────────────────────────────────────────────────────────────────────────────
# EXTRACT BEST PARAMETER SET RESULTS
# ────────────────────────────────────────────────────────────────────────────
# The optimization returns the parameter set that achieved the highest Sharpe ratio

best_atr    = stats_best._strategy.atr_stop      # Optimal ATR stop multiplier
best_ret    = stats_best["Return [%]"]           # Total return with best parameters
best_sharpe = stats_best["Sharpe Ratio"]         # Best achieved Sharpe ratio

# ────────────────────────────────────────────────────────────────────────────
# DISPLAY OPTIMIZATION RESULTS
# ────────────────────────────────────────────────────────────────────────────
print("=" * 50)
print("OPTIMIZATION RESULTS - BEST PARAMETER SET")
print("=" * 50)
print(f"Optimal ATR Stop Multiplier : {best_atr:.2f}")
print(f"Total Return [%]            : {best_ret:.2f}%")
print(f"Sharpe Ratio                : {best_sharpe:.2f}")
print("=" * 50)
print(f"Interpretation:")
print(f"- ATR Stop = {best_atr:.2f} means stop loss is set at {best_atr:.2f} * ATR below/above entry")
print(f"- Higher values = wider stops (less frequent but larger losses)")
print(f"- Lower values = tighter stops (more frequent but smaller losses)")

C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/7 [00:00<?, ?it/s]

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. 

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

=== Best parameter set ===
atr_stop      : 1.00
Return [%]    : 149.31
Sharpe Ratio  : 1.70


C:\Users\Serkan POLAT\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


In [ ]:
# ============================================================================
# COMPREHENSIVE OPTIMIZATION RESULTS ANALYSIS
# ============================================================================
# Analyze all parameter combinations tested during optimization to understand
# the relationship between ATR stop settings and strategy performance
# ============================================================================

# ────────────────────────────────────────────────────────────────────────────
# PROCESS OPTIMIZATION HEATMAP DATA
# ────────────────────────────────────────────────────────────────────────────
# The heatmap contains Sharpe ratios for all tested parameter combinations
# Convert to a more readable format for analysis

summary = (
    heatmap
    .rename("Sharpe Ratio")                   # Convert Series to named column
    .reset_index()                            # Move atr_stop from index to column
    .rename(columns={"index": "atr_stop"})    # Rename index column
    .sort_values("atr_stop")                  # Sort by ATR stop value for readability
)

# ────────────────────────────────────────────────────────────────────────────
# COLLECT ADDITIONAL PERFORMANCE METRICS
# ────────────────────────────────────────────────────────────────────────────
# The heatmap only contains the optimization metric (Sharpe Ratio)
# Run each parameter combination again to collect total returns

print("Collecting detailed performance metrics for all parameter combinations...")

returns = []
for val in summary["atr_stop"]:
    # Run backtest with specific ATR stop value
    res = bt.run(atr_stop=val)
    returns.append(res["Return [%]"])

# Add returns to summary table
summary["Return [%]"] = returns

# ────────────────────────────────────────────────────────────────────────────
# DISPLAY COMPREHENSIVE RESULTS TABLE
# ────────────────────────────────────────────────────────────────────────────
print("\n" + "=" * 70)
print("COMPLETE OPTIMIZATION SWEEP RESULTS")
print("=" * 70)
print("ATR Stop | Sharpe Ratio | Total Return | Performance Notes")
print("-" * 70)

# Display results with performance interpretation
for _, row in summary.iterrows():
    atr_stop = row["atr_stop"]
    sharpe = row["Sharpe Ratio"]
    ret = row["Return [%]"]
    
    # Add performance notes based on metrics
    if sharpe > 1.0:
        note = "Excellent risk-adjusted return"
    elif sharpe > 0.5:
        note = "Good risk-adjusted return"
    elif sharpe > 0:
        note = "Positive but moderate performance"
    else:
        note = "Poor performance"
    
    print(f"{atr_stop:8.2f} | {sharpe:11.2f} | {ret:11.2f}% | {note}")

print("\n" + "=" * 70)
print("ANALYSIS SUMMARY:")
print("=" * 70)
print("• Sharpe Ratio > 1.0: Excellent risk-adjusted returns")
print("• Sharpe Ratio 0.5-1.0: Good risk-adjusted returns") 
print("• Sharpe Ratio 0-0.5: Acceptable but suboptimal")
print("• Sharpe Ratio < 0: Strategy underperformed risk-free rate")
print("\nOptimal parameter selection should balance:")
print("- High Sharpe ratio (risk-adjusted returns)")
print("- Acceptable total returns")
print("- Reasonable drawdown characteristics")

# Store results for further analysis
print(f"\nBest parameter: ATR Stop = {summary.loc[summary['Sharpe Ratio'].idxmax(), 'atr_stop']:.2f}")
print(f"Worst parameter: ATR Stop = {summary.loc[summary['Sharpe Ratio'].idxmin(), 'atr_stop']:.2f}")

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_17488\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_17488\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]

C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_17488\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


Backtest.run:   0%|          | 0/3985 [00:00<?, ?bar/s]


=== Full sweep results ===
 atr_stop  Sharpe Ratio  Return [%]
     1.00      1.703812  149.305559
     1.25      1.623712  135.272356
     1.50      1.610710  138.235143
     1.75      1.467931  113.844691
     2.00      1.487782  121.810183
     2.25      1.550615  129.988240
     2.50      1.527503  127.635745


C:\Users\Serkan POLAT\AppData\Local\Temp\ipykernel_17488\3454991667.py:15: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  res = bt.run(atr_stop=val)


In [ ]:
# ============================================================================
# FINAL STRATEGY VISUALIZATION
# ============================================================================
# Generate comprehensive visual analysis of the optimized VWAP breakout strategy
# including price action, trade signals, equity curve, and drawdown analysis
# ============================================================================

# Display interactive backtesting results with optimal parameters
# This plot includes:
# - Price chart with VWAP overlay
# - Buy/sell signal markers  
# - Equity curve showing portfolio growth
# - Drawdown periods highlighted
# - Trade statistics and performance metrics

bt.plot()

GridPlot(id='p1782', ...)